In [2]:
from flask import Flask, request, jsonify
import time
import nest_asyncio
import threading
import redis

# Fix Jupyter event loop
nest_asyncio.apply()

# Redis client
r = redis.StrictRedis(host='localhost', port=6379, db=0, decode_responses=True)

# Flask app
app = Flask(__name__)

# Config
RATE_LIMIT = 5
TIME_WINDOW = 60  # seconds

# Rate limit check using Redis
def is_rate_limited(client_id):
    now = int(time.time())
    window_start = now - TIME_WINDOW
    key = f"rate_limit:{client_id}"

    # Fetch all request timestamps
    request_times = r.lrange(key, 0, -1)
    request_times = [int(t) for t in request_times if int(t) > window_start]

    # Update the Redis list
    r.delete(key)
    if request_times:
        r.rpush(key, *request_times)

    if len(request_times) >= RATE_LIMIT:
        return True

    # Add current request time
    r.rpush(key, now)
    r.expire(key, TIME_WINDOW)
    return False

# Define API endpoint
@app.route("/api/data", methods=["GET"])
def get_data():
    client_ip = request.remote_addr or "test_user"
    if is_rate_limited(client_ip):
        return jsonify({"error": "Rate limit exceeded. Try again later."}), 429
    return jsonify({"data": "Here’s your awesome API response!"})

# Run Flask in background thread
def run_app():
    app.run(port=5000)

thread = threading.Thread(target=run_app)
thread.daemon = True
thread.start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [4]:
import requests

for i in range(7):
    res = requests.get("http://127.0.0.1:5000/api/data")
    try:
        print(f"[{i+1}] Status: {res.status_code} - {res.json()}")
    except:
        print(f"[{i+1}] Status: {res.status_code} - Non-JSON response")

[1] Status: 200 - {'data': 'Here’s your awesome API response!'}
[2] Status: 200 - {'data': 'Here’s your awesome API response!'}
[3] Status: 200 - {'data': 'Here’s your awesome API response!'}
[4] Status: 200 - {'data': 'Here’s your awesome API response!'}
[5] Status: 200 - {'data': 'Here’s your awesome API response!'}
[6] Status: 429 - {'error': 'Rate limit exceeded. Try again later.'}
[7] Status: 429 - {'error': 'Rate limit exceeded. Try again later.'}
